# 最終課題

### 課題内容
『武蔵野大学のWebサイトのサイトマップを抽出する』
・武蔵野大学Webサイトのトップページにアクセス
・同一ドメインの全てのリンク（コメントアウトされていないもの）を辿り，全ページのURLと<title>を辞書型変数に格納する
key：URL
value：<title></title>で挟まれた文字列
・辞書型変数を print() で表示する


・注意
Webサイトにアクセスする際は必ず time.sleep(秒数) を使って負荷軽減をすること
課題用のipynb形式のファイルを作成し，githubフローでファイルの変更履歴を管理すること


In [5]:
#　インポート
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
url = "https://www.musashino-u.ac.jp/"
# クロール対象をこのドメイン内に限定するための文字列。
TARGET_DOMAIN = "musashino-u.ac.jp"
sitemap = {}    # サイトマップ用の辞書を作成


In [ ]:
res = requests.get(url)
time.sleep(1) # アクセス負荷軽減
soup = BeautifulSoup(res.text, 'html.parser')


In [10]:
# --- トップページへのアクセスと解析 ---
print("ステップ1: トップページにアクセスし、タイトルを取得します。")
time.sleep(1)
# 取得したページの文字コードを自動判別して設定する（文字化けを防ぐため）
res.encoding = res.apparent_encoding

    # --- ページタイトルの取得と格納 ---
    # soup.find('title')を使って、HTMLの中から<title>タグを1つ見つけ出す。
title_tag = soup.find('title')

# .textを使ってタグの中の文字列を取得し、.strip()で余分な空白を削除する。
# titleタグが見つかった場合のみ、この処理を行う。
if title_tag:
    page_title = title_tag.text.strip()
else:
    page_title = "タイトルなし"

# 辞書に、キーをURL、バリューをページタイトルとして格納する。
sitemap[url] = page_title

# --- 結果の表示 ---
# 現時点でのsitemap辞書の中身を表示して、正しく取得できたか確認する。
print("\n--- 取得結果 ---")
print(sitemap)

ステップ1: トップページにアクセスし、タイトルを取得します。

--- 取得結果 ---
{'https://www.musashino-u.ac.jp/': '武蔵野大学'}
